In [ ]:
import random

class Load:
    # For now, sinusoidal time-varying server load model with peak in the afternoon.
    # Load between 0 and 1, in 2-hour intervals starting at 0:00-1:59.
    load_by_hours = [
        0.4, 0.4, 0.4, 0.4, 0.6, 0.8, 1.0, 1.0, 1.0, 0.8, 0.6, 0.6
    ]

    @staticmethod
    def at_hour(hour):
        assert 0 <= hour < 24
        return max(0, min(Load.load_by_hours[hour // 2] + random.uniform(-0.1, 0.1), 1))

class Emissions:
    # Average direct (not including LCA) emissions factors in g CO2e per kWh
    # for electricity consumed in Switzerland for each pair of hours starting
    # with 0:00-1:59, based on hourly data for 2023 from https://electricitymaps.com/.
    gco2_per_kwh_by_hours = [
        63, 58, 58, 52, 49, 47, 45, 45, 49, 54, 62, 63,
    ]

    @staticmethod
    def at_hour(hour, kwh):
        return Emissions.gco2_per_kwh_by_hours[hour // 2] * kwh

    @staticmethod
    def average():
        return sum(Emissions.gco2_per_kwh_by_hours) / 12

class Server:
    def __init__(self, power_w):
        self.power_w = power_w

    def power_at_hour(self, hour):
        l = Load.at_hour(hour)
        return self.power_w * (0.6 * l + 0.4 * l * l)

    def energy_and_emissions(self, start_hour, end_hour):
        # Integrate power and emissions over time based on load and emissions models.
        assert 0 <= start_hour < 24 and 0 <= end_hour < 24 and start_hour < end_hour
        wh, gco2e = 0, 0
        for hour in range(start_hour, end_hour + 1):
            p = self.power_at_hour(hour)
            wh += p
            gco2e += Emissions.at_hour(hour, p / 1000)
        return wh, gco2e

class NetworkGear:
    def __init__(self, power_w):
        self.power_w = power_w

    def energy_and_emissions(self, start_hour, end_hour):
        assert 0 <= start_hour < 24 and 0 <= end_hour < 24 and start_hour < end_hour
        # No load factor because power utilization at >10% load is ~= power utilization at 100% load.
        wh = self.power_w * (end_hour - start_hour + 1)
        gco2e = 0
        for hour in range(start_hour, end_hour + 1):
            gco2e += Emissions.at_hour(hour, self.power_w / 1000)
        return wh, gco2e

class DataCenter:
    def __init__(self, servers, network_gear, cooling_power_kw, lighting_power_kw):
        self.servers = servers
        self.network_gear = network_gear
        self.cooling_power_kw = cooling_power_kw
        self.lighting_power_kw = lighting_power_kw

    def energy_and_emissions(self, start_hour, end_hour):
        assert 0 <= start_hour < 24 and 0 <= end_hour < 24 and start_hour < end_hour

        self.compute_kwh, compute_gco2e = 0, 0
        for server in self.servers:
            energy, emissions = server.energy_and_emissions(start_hour, end_hour)
            self.compute_kwh += energy
            compute_gco2e += emissions
        self.compute_kwh /= 1000  # Convert Wh to kWh

        network_kwh, network_gco2e = 0, 0
        for gear in self.network_gear:
            energy, emissions = gear.energy_and_emissions(start_hour, end_hour)
            network_kwh += energy
            network_gco2e += emissions
        network_kwh /= 1000  # Convert Wh to kWh

        cooling_kwh = self.cooling_power_kw * (end_hour - start_hour + 1)
        cooling_gco2e = Emissions.average() * cooling_kwh
        lighting_kwh = self.lighting_power_kw * (end_hour - start_hour + 1)
        lighting_gco2e = Emissions.average() * lighting_kwh

        self.total_kwh = self.compute_kwh + network_kwh + cooling_kwh + lighting_kwh
        self.total_kgco2e = (compute_gco2e + network_gco2e + cooling_gco2e + lighting_gco2e) / 1000
        return self.total_kwh, self.total_kgco2e

    def pue(self):
        return self.total_kwh / self.compute_kwh if self.compute_kwh > 0 else float('inf')

# Example usage
servers = (
    [Server(100) for _ in range(1000)] +
    [Server(500) for _ in range(100)] +
    [Server(1000) for _ in range(20)]
)
network_gear = (
    [NetworkGear(70) for _ in range(25)] +
    [NetworkGear(90) for _ in range(5)]
)

cooling_power_kw = 20  # Cooling system power consumption in kW
lighting_power_kw = 1  # Lighting power consumption in kW
data_center = DataCenter(servers, network_gear, cooling_power_kw, lighting_power_kw)
total_energy, total_emissions = data_center.energy_and_emissions(0, 23)
pue = data_center.pue()

print(f"Total number of servers: {len(servers)}")
print(f"Total energy consumption for one day: {total_energy:.2f} kWh")
print(f"Total emissions for one day: {total_emissions:.2f} kg CO2e")
print(f"PUE: {pue:.2f}")


Total number of servers: 1120
Total energy consumption for one day: 2971.29 kWh
Total emissions for one day: 154.61 kg CO2e
PUE: 1.23
53.75
